In [57]:
import random as ra
import pandas as pd
import numpy as np
import datetime as dtime
import matplotlib.pyplot as plt
import sqlite3

records, emissions = 5000, 200
indexes = np.random.randint(0, records - 1, emissions)
limits = [[10, 11], range(160, 171), None,
          range(60, 71), list(range(1, 8)) + list(range(121, 131))]
indexes

array([1805, 4221, 2389, 3391, 1371,  373, 3793, 1620, 2453, 1755, 1882,
       1965, 3581, 1024,  529,  723,  627,  835, 3843, 2880, 4345, 2210,
       2769, 3677, 1874,  914, 4625, 1262,  187, 3053, 2973, 1581, 2826,
       2800, 3937,  345, 2480, 4267, 3108, 3180, 3421, 2399, 1918, 3359,
       2310, 3748, 3320,  267, 3794, 2931, 1004, 1907, 3306,  612, 4245,
        997, 3351, 1163, 1151, 2802, 4238, 4474, 4785, 1657, 3993, 2715,
       4633, 1900, 3577,  309, 2672,   18,   99, 3892, 1200, 2876, 1042,
       3758, 1004, 3599, 1165, 3125,  270,  189, 1874, 1382, 1071, 3636,
       3637, 2173, 2809, 1554, 1998,  367, 3807, 4052, 1872, 2307,  852,
        280,  949, 4824, 1426, 4618,  310,  501,  468, 2085, 1940, 2328,
       2082, 2958, 4637, 4223, 3570, 2454, 3702, 1685,  552, 4276, 1421,
       4630, 4649, 3979, 3150, 3777, 4740, 3162, 3088, 1340, 3137, 1483,
       2720, 3072, 4668,  237, 2812, 4164, 4188, 3810, 1249, 2507, 2004,
        863, 4011,  914, 2909,   63, 4812,  532, 12

In [58]:
data = pd.DataFrame(columns=['time', 'dynamic_range', 'viewing_angle',
                             'focal_length', 'temperature', 'oscillation_frequency'])

def generate_dates(start, passing, amount):
    current = start
    while amount >= 0:
        current = current + dtime.timedelta(days=passing)
        yield current
        amount -= 1

start_date = dtime.datetime(2009, 7, 15)
dates = []
for i in generate_dates(start_date, 1, records - 1):
    dates.append(i.strftime("%d.%m.%Y"))
    
data['time'] = dates
data

,time,dynamic_range,viewing_angle,focal_length,temperature,oscillation_frequency
0,16.07.2009,NaN,NaN,NaN,NaN,NaN
1,17.07.2009,NaN,NaN,NaN,NaN,NaN
2,18.07.2009,NaN,NaN,NaN,NaN,NaN
3,19.07.2009,NaN,NaN,NaN,NaN,NaN
4,20.07.2009,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4995,20.03.2023,NaN,NaN,NaN,NaN,NaN
4996,21.03.2023,NaN,NaN,NaN,NaN,NaN
4997,22.03.2023,NaN,NaN,NaN,NaN,NaN
4998,23.03.2023,NaN,NaN,NaN,NaN,NaN


In [59]:
data['dynamic_range'] = np.random.randint(4, 9, records)
data['viewing_angle'] = np.random.randint(6, 160, records)
data['focal_length'] = np.random.uniform(2.8, 16, records)
data['temperature'] = np.random.randint(-10, 60, records)
data['oscillation_frequency'] = np.random.randint(8, 120, records)

for col, limit in zip(data.columns[1:], limits):
    for i in indexes:
        try:
            data.loc[i, col] = ra.choice(limit)
        except TypeError:
            data.loc[i, col] = ra.uniform(0.2, 2.8)

data

,time,dynamic_range,viewing_angle,focal_length,temperature,oscillation_frequency
0,16.07.2009,8,76,12.130096,48,75
1,17.07.2009,7,133,9.612135,45,103
2,18.07.2009,7,63,11.103212,54,23
3,19.07.2009,6,129,10.750230,56,71
4,20.07.2009,8,122,14.644767,21,14
...,...,...,...,...,...,...
4995,20.03.2023,5,120,15.882606,43,66
4996,21.03.2023,6,106,10.463586,43,68
4997,22.03.2023,6,115,7.334849,-2,57
4998,23.03.2023,6,54,10.377594,53,22


In [60]:
conn = sqlite3.connect('main_database.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS readings (time TIMESTAMP, dynamic_range INT,
         viewing_angle INT, focal_length NUMERIC, temperature INT, oscillation_frequency INT)''')
conn.commit()

data.to_sql('readings', conn, if_exists='replace', index = False)
c.execute('SELECT * FROM readings')
for row in c.fetchall():
    print(row)

data.to_excel('database.xlsx', index=False)

('16.07.2009', 8, 76, 12.13009633760397, 48, 75)
('17.07.2009', 7, 133, 9.612134723106063, 45, 103)
('18.07.2009', 7, 63, 11.103211684631528, 54, 23)
('19.07.2009', 6, 129, 10.750230055893404, 56, 71)
('20.07.2009', 8, 122, 14.644767478072612, 21, 14)
('21.07.2009', 8, 20, 11.495282273732439, 33, 68)
('22.07.2009', 4, 110, 9.305539857811358, 14, 34)
('23.07.2009', 4, 39, 9.514850497685089, -10, 12)
('24.07.2009', 8, 112, 15.712197745715347, -4, 46)
('25.07.2009', 8, 141, 10.707476377256993, 38, 53)
('26.07.2009', 8, 46, 9.141735666452458, 52, 75)
('27.07.2009', 8, 19, 12.87242599028573, 6, 30)
('28.07.2009', 5, 120, 8.618466057063372, -2, 93)
('29.07.2009', 7, 104, 7.611269149615822, -2, 33)
('30.07.2009', 4, 59, 4.571845105349911, 36, 14)
('31.07.2009', 8, 36, 7.195143733640016, 28, 25)
('01.08.2009', 6, 64, 6.017537283858649, 50, 52)
('02.08.2009', 5, 152, 7.191826908950508, 53, 38)
('03.08.2009', 11, 170, 0.6271562724783885, 67, 125)
('04.08.2009', 7, 9, 2.8027873871413385, -5, 105)

('10.01.2015', 10, 166, 1.2475898696751344, 65, 121)
('11.01.2015', 8, 58, 12.865471181049045, 41, 70)
('12.01.2015', 5, 133, 7.124194873610466, 32, 72)
('13.01.2015', 4, 159, 13.812910671905517, 22, 51)
('14.01.2015', 10, 169, 1.1921491923758791, 69, 125)
('15.01.2015', 7, 6, 4.436497437511212, 36, 64)
('16.01.2015', 6, 12, 13.403014288522293, 12, 102)
('17.01.2015', 6, 149, 11.947251975914803, 19, 84)
('18.01.2015', 7, 16, 5.665914192796724, 36, 68)
('19.01.2015', 4, 56, 5.358720318869694, 48, 23)
('20.01.2015', 4, 97, 7.30742126943357, 28, 26)
('21.01.2015', 6, 59, 5.993640678078111, -6, 76)
('22.01.2015', 7, 82, 13.669142731410417, 12, 75)
('23.01.2015', 8, 20, 13.583134267173893, 2, 107)
('24.01.2015', 7, 25, 9.503303446710657, 47, 85)
('25.01.2015', 7, 78, 12.726544279069945, 41, 119)
('26.01.2015', 4, 41, 6.8401897089687695, -6, 77)
('27.01.2015', 5, 51, 9.576952684320476, 49, 55)
('28.01.2015', 4, 157, 9.298341511225463, 58, 28)
('29.01.2015', 6, 104, 7.050269780875223, 27, 117

('18.02.2019', 4, 50, 12.58015816271676, 14, 58)
('19.02.2019', 7, 112, 4.726339063926492, 27, 72)
('20.02.2019', 6, 64, 7.0742706513261, -6, 80)
('21.02.2019', 4, 41, 7.950854909351151, 49, 17)
('22.02.2019', 4, 150, 11.810192540206192, 32, 113)
('23.02.2019', 6, 49, 13.415911816959767, 25, 119)
('24.02.2019', 5, 19, 10.056670673242213, 25, 20)
('25.02.2019', 8, 59, 3.2843665140428513, 43, 57)
('26.02.2019', 5, 156, 7.870910227336949, 33, 27)
('27.02.2019', 7, 130, 10.213488914392592, 15, 39)
('28.02.2019', 5, 15, 13.571365716057237, 42, 110)
('01.03.2019', 4, 7, 9.590950693135644, 50, 16)
('02.03.2019', 8, 20, 14.974513010371275, -9, 105)
('03.03.2019', 6, 81, 9.052286003732391, 35, 18)
('04.03.2019', 6, 18, 12.70646957700393, 51, 16)
('05.03.2019', 5, 99, 7.466562224960658, 25, 70)
('06.03.2019', 4, 8, 10.098078915447157, 56, 111)
('07.03.2019', 5, 83, 5.37594350082645, 23, 24)
('08.03.2019', 8, 108, 6.328559936358056, -1, 93)
('09.03.2019', 8, 148, 4.672373428354599, 15, 32)
('10.0